# Volc_DEM
9/1/2022
This notebook reads in DEM data for the cascade volcanoes, visualizes the data, and stores it for later use

Import Modules

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import Figure
import numpy as np
import rasterio as rio
from rasterio.plot import show
from rasterio.merge import merge
from pathlib import Path
import richdem as rd
from mpl_toolkits import mplot3d

In [ ]:
pi = np.pi
color_scheme = 'jet'
radius = 6371e3

In [ ]:
volc_lat_lon = {}
volc_lat_lon['Mt_Rainier'] = [46.8528857, -121.7603744, 4392.5, 46.625, -122.000,[]]
volc_lat_lon['Mt_Adams'] = [46.202621, -121.4906384, 3743.2, 46.125, -121.625,[]]
volc_lat_lon['Mt_Baker'] = [48.7773426,  -121.8132008, 3287.6, 48.750, -121.875,[]]
# change the lat and lon of mt st helens to the middle of the dome instead of the highest point
volc_lat_lon['Mt_St_Helens'] =[46.200472222222224,-122.18883611111112,2549, 46.125, -122.250,[]] #[46.1912, -122.1944, 2549]
volc_lat_lon['Glacier_Peak'] = [48.1112273, -121.1139922, 3213, 48, -121.25,[]]
volc_lat_lon['Crater_Lake']=[42.907745, -122.143494, 1883, 42, -123,[]]
volc_lat_lon['Mt_Hood']=[45.373221, -121.696509, 3428.7, 45, -122,[]]
volc_lat_lon['Newberry']=[43.7220653, -121.2344654, 2435, 43, -122,[]]

In [ ]:
# define function to get the distance of the lower-left corner to the center of the volcano
def distance(slat, slon, elat, elon, radius):
    lam = slon-elon
    g = (np.sin(slat)*np.sin(elat))+(np.cos(slat)*np.cos(elat)*np.cos(lam))
    d = radius*np.arccos(g)
    dist = d/10
    return dist

In [ ]:
# vertical distance
for name in volc_lat_lon:
    slat = np.deg2rad(volc_lat_lon[name][0])
    elat = np.deg2rad(volc_lat_lon[name][3])
    slon = np.deg2rad(volc_lat_lon[name][4])
    elon = np.deg2rad(volc_lat_lon[name][4])
    v_index = distance(slat, slon, elat, elon, radius)
    volc_lat_lon[name][5].append(v_index)

# horizontal distances
for name in volc_lat_lon:
    slat = np.deg2rad(volc_lat_lon[name][3])
    elat = np.deg2rad(volc_lat_lon[name][3])
    slon = np.deg2rad(volc_lat_lon[name][1])
    elon = np.deg2rad(volc_lat_lon[name][4])
    h_index = distance(slat, slon, elat, elon, radius)
    volc_lat_lon[name][5].append(h_index)

Merging together washington Data

In [ ]:
volc_names = ['Mt_Rainier','Mt_St_Helens','Mt_Adams','Mt_Baker','Glacier_Peak']

In [ ]:
for name in volc_names:
    path = Path('DEM_data/'+name)
    path1 = Path('DEM_data/'+name+'/.ipynb_checkpoints')
    #path.mkdir(parents=True, exist_ok=True)
    out_path = 'DEM_data/'+str(name)+'/mosaic_'+str(name)+'.tif'
    volc_files = list(path.iterdir())
    to_mosiac = []
    for d in volc_files:
        if d != path1:
            raster = rio.open(d)
            to_mosiac.append(raster)
    mosaic, output = merge(to_mosiac)
    output_meta = raster.meta.copy()
    output_meta.update(
        {'driver': 'GTiff',
            'height': mosaic.shape[1],
            'width': mosaic.shape[2],
            'transform': output,})
    with rio.open(out_path, 'w', **output_meta) as m:
        m.write(mosaic)
    break

In [ ]:
dem = rio.open('DEM_data/Mt_Rainier/mosaic_Mt_Rainier.tif')
dem_array = dem.read(1).astype('float64')

In [ ]:
volc = rd.rdarray(dem_array[100:4101,100:2820], no_data=-9999)
#slope = rd.TerrainAttribute(volc,attrib = 'slope_riserun')
#aspect = rd.TerrainAttribute(volc, attrib = 'aspect')
fig1  = rd.rdShow(volc, axes=True, cmap=color_scheme, figsize=(15,8))
# slope = rd.TerrainAttribute(volc,attrib = 'slope_riserun')
# aspect = rd.TerrainAttribute(volc, attrib = 'aspect')
# fig2 = rd.rdShow(slope, axes=False, cmap=color_scheme, figsize=(15,8))
# fig3 = rd.rdShow(aspect, axes=False, cmap=color_scheme, figsize=(15,8))


Plot the data using matplotlib

In [ ]:
y_index = dem_array.shape[0]-volc_lat_lon['Mt_Rainier'][5][0]
x_index = volc_lat_lon['Mt_Rainier'][5][1]

fig, ax = plt.subplots(1,2, figsize=(20,10))
b = ax[0].contourf(volc,200,cmap = 'bone_r')
ax[0].scatter(x_index, y_index,s=200,marker='*',c='r')
ax[0].invert_yaxis()
fig.colorbar(b)

# fig, ax = plt.subplots(1,1, figsize=(10,15))
# a = ax.contourf(aspect, 200, cmap = 'jet')
# #ax.scatter(1666,873,s=100,marker='*',c='r')
# ax.invert_yaxis()
# fig.colorbar(a)


lats = [46.98832, 46.988131,46.899719,46.731319,46.859032,46.786442,46.929694,46.841649,46.90306,46.867092,
46.96633,46.941399,46.750599,46.750599,46.83564,46.81905,46.850849,46.613279]
lons = [-122.00541,-121.961063,-121.815331,-121.571171,-121.642592,-121.742149,-121.988976,-121.948912,-121.94439,
-121.953246,-121.9834,-121.670998,-121.809601,-121.809601,-121.732979,-121.842133,-121.792953,-121.404485]
stas = ['CARB','CRBN','OBSR','OPCH','PANH','PARA','PR04','PR05','RUSH','SIFT','VOIT','FMW','LO2','LON','RCM','RER',
'STAR','WPEQ']

for i in range(0,len(lats)):
    v = distance(np.deg2rad(46.625), np.deg2rad(lons[i]), np.deg2rad(lats[i]), np.deg2rad(lons[i]), radius)
    h_dist = distance(np.deg2rad(lats[i]),np.deg2rad(-122.000),np.deg2rad(lats[i]), np.deg2rad(lons[i]), radius)
    v_dist = dem_array.shape[0]-v
    if lats[i] < 46.625:
        v_dist = dem_array.shape[0] + v
    if lons[i] < -122.000:
        h_dist = -1*h_dist
    ax[0].scatter(h_dist,v_dist,s=100)
    ax[0].text(h_dist,v_dist,stas[i])
    ax[1].scatter(lons[i],lats[i],s = 100, c = 'k')
    ax[1].text(lons[i],lats[i],stas[i])
# ax = plt.subplot(1,2,1, polar=True)
# a = ax.contourf(aspect, 10, cmap = 'jet')
# #ax[0].invert_yaxis()
# fig.colorbar(a)

# ax.set_theta_offset(pi/2)
# ax.set_theta_direction(-1)
# for i in range(0,len(r)):
#     ax.plot(np.deg2rad(theta[i]),r[i], 'g.')
#  #plot the estimated direction of the event
# # for rad in rads:
# #     ax.plot(rad, az_thr, 'b.', markersize = 2)
# plt.show()
# x = np.array(range(300))
# ax.plot(x, x, ls='-', linewidth=2, color='k')

# fig.colorbar(a)